In [8]:
using JuMP
using GLPK

In [85]:
hub_opening_costs = [40, 40, 40, 40, 40]
consumer_demand = [10, 8, 12, 16, 9]
hab_capacity = [6, 2, 4, 5, 7]
hub_to_consumer_delivery_costs = [
    30  15  59  78  27;
    50  42  25  30  53;
    64  14  30  20  62;
    46  19  66  48  11;
    19  40  60  31  27
]

hub_count = size(hub_to_consumer_delivery_costs, 1)
consumer_count = size(hub_to_consumer_delivery_costs, 2)

5

In [86]:
if size(hub_opening_costs, 1) != hub_count
    throw(ErrorException("The size of the first dimension does not match the expected hub count"))
end

In [87]:
facility_model = Model(GLPK.Optimizer)

# open or not open the hub
@variable(facility_model, y[1:hub_count], binary=true)
# The share of demand met by delivery from hub to consumer.
@variable(
    facility_model, 
    0 <= x[1:hub_count, 1:consumer_count] <= 1
)

for f=1:hub_count
    for c=1:consumer_count
        @constraint(facility_model, x[f,c] <= y[f])
    end
end

for f=1:hub_count
    for c=1:consumer_count
        @constraint(facility_model, x[f,c] <= hab_capacity[f]/consumer_demand[c])
    end
end

for c=1:consumer_count
    @constraint(facility_model, sum([x[f,c] for f in 1:hub_count]) == 1)
end

@objective(
    facility_model,
    Min,
    sum(hub_opening_costs[f] * y[f] for f in 1:hub_count) + 
    sum(hub_to_consumer_delivery_costs[f,c] * x[f,c] for f in 1:hub_count, c in 1:consumer_count)
)

optimize!(facility_model)

In [88]:
println(JuMP.objective_value(facility_model))
println("Opened hubs: ", value.(y))
println("Assignments x[f,c]:")
for f in 1:hub_count, c in 1:consumer_count
    print(value(x[f,c]), " ")
    if c == consumer_count; println(); end
end

265.7736111111111
Opened hubs: [0.0, 0.0, 1.0, 1.0, 1.0]
Assignments x[f,c]:
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.5 0.3333333333333333 0.25 0.0 
0.30000000000000004 0.5 0.08333333333333331 0.3125 0.5555555555555556 
0.7 0.0 0.5833333333333334 0.4375 0.4444444444444444 
